Задача: выполнить решение задачи по определению оценок

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Embedding, LSTM, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# nltk.download('stopwords')

In [2]:
file_path = 'train.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.head()

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5
3,003885A45F42,The best time in life is when you become yours...,4.5,4.5,4.5,4.5,4.0,5.0
4,0049B1DF5CCC,Small act of kindness can impact in other peop...,2.5,3.0,3.0,3.0,2.5,2.5


In [5]:
X = df['full_text']

In [6]:
Y = df[['cohesion','syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']]

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [8]:
text_data = X_train.astype(str).tolist()

In [9]:
text_data_test = X_test.astype(str).tolist()

In [10]:
stop_words = set(stopwords.words('english'))

In [11]:
stop_words.remove('not')
stop_words.remove('no')

In [12]:
max_len = 100

In [13]:
tokenizer = Tokenizer(oov_token="<OOV>")

In [14]:
tokenizer.fit_on_texts(text_data)

In [15]:
sequences = tokenizer.texts_to_sequences(text_data)

In [16]:
train_texts = pad_sequences(sequences, maxlen=max_len)

In [17]:
test_texts = tokenizer.texts_to_sequences(text_data_test)

In [18]:
test_texts = pad_sequences(test_texts, maxlen=max_len)

In [19]:
input_text = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128)(input_text)
lstm_layer = LSTM(128)(embedding_layer)
dense_layer = Dense(64, activation= 'relu')(lstm_layer)

# отдельные выходные слои для каждой оценки
output1 = Dense(1, activation='linear', name='cohesion')(dense_layer)
output2 = Dense(1, activation='linear', name='syntax')(dense_layer)
output3 = Dense(1, activation='linear', name='vocabulary')(dense_layer)
output4 = Dense(1, activation='linear', name='phraseology')(dense_layer)
output5 = Dense(1, activation='linear', name='grammar')(dense_layer)
output6 = Dense(1, activation='linear', name='conventions')(dense_layer)

In [20]:
model = Model(inputs=input_text, outputs=[output1, output2, output3, output4, output5, output6])

In [21]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 128)     2500352     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    (None, 128)          131584      ['embedding[0][0]']              
                                                                                                  
 dense (Dense)                  (None, 64)           8256        ['lstm[0][0]']                   
                                                                                              

In [22]:
model.compile(optimizer='adam', loss={'cohesion': 'mean_squared_error',
                                     'syntax': 'mean_squared_error',
                                     'vocabulary': 'mean_squared_error',
                                     'phraseology': 'mean_squared_error',
                                     'grammar': 'mean_squared_error',
                                     'conventions': 'mean_squared_error'})

In [23]:
model.fit(train_texts, {'cohesion': Y_train['cohesion'], 'syntax': Y_train['syntax'], 'vocabulary': Y_train['vocabulary'], 'phraseology': Y_train['phraseology'], 'grammar': Y_train['grammar'], 'conventions': Y_train['conventions']}, epochs=10, batch_size=64)

Epoch 1/10
49/49 [==============================] - 15s 239ms/step - loss: 13.4770 - cohesion_loss: 2.3193 - syntax_loss: 2.0947 - vocabulary_loss: 2.3128 - phraseology_loss: 2.2078 - grammar_loss: 2.1869 - conventions_loss: 2.3554
Epoch 2/10
49/49 [==============================] - 15s 315ms/step - loss: 2.5916 - cohesion_loss: 0.4513 - syntax_loss: 0.4120 - vocabulary_loss: 0.3459 - phraseology_loss: 0.4295 - grammar_loss: 0.4925 - conventions_loss: 0.4604
Epoch 3/10
49/49 [==============================] - 13s 265ms/step - loss: 2.5215 - cohesion_loss: 0.4342 - syntax_loss: 0.4024 - vocabulary_loss: 0.3358 - phraseology_loss: 0.4196 - grammar_loss: 0.4829 - conventions_loss: 0.4466
Epoch 4/10
49/49 [==============================] - 12s 238ms/step - loss: 2.3950 - cohesion_loss: 0.4126 - syntax_loss: 0.3803 - vocabulary_loss: 0.3158 - phraseology_loss: 0.4015 - grammar_loss: 0.4606 - conventions_loss: 0.4243
Epoch 5/10
49/49 [==============================] - 13s 267ms/step - loss: 

In [24]:
# оценка модели на проверочном множестве
loss = model.evaluate(test_texts, {'cohesion': Y_test['cohesion'], 'syntax': Y_test['syntax'], 'vocabulary': Y_test['vocabulary'], 'phraseology': Y_test['phraseology'], 'grammar': Y_test['grammar'], 'conventions': Y_test['conventions']})

25/25 [==============================] - 2s 36ms/step - loss: 2.9152 - cohesion_loss: 0.4848 - syntax_loss: 0.4810 - vocabulary_loss: 0.3822 - phraseology_loss: 0.4996 - grammar_loss: 0.5679 - conventions_loss: 0.4997
